In [1]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import display
from loguru import logger
from phd_visualizations.utils.units import unit_conversion

# Setup environment for running MATLAB code if not done externally
import os

os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

filename_process_data = '20231030_solarMED.csv'
filename_process_data2 = '20231030_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '60s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

# Load variables information
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Load plot configuration
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(data_path / filename_process_data, parse_dates=True, index_col='TimeStamp')
# Rename index column to "time"
df.index.names = ['time']
# Set UTC timezone
df = df.tz_localize('UTC')

# TEMP: Read additional MED data from librescada logging, in the future everything should be read directly from librescada
df_aux = pd.read_csv(data_path / filename_process_data2, parse_dates=True, index_col='time')
# Set UTC timezone
df_aux = df_aux.tz_localize('UTC')
# Add columns from df_aux to df
df = pd.concat([df, df_aux], axis=1)

display(df.head())

# Preprocessing
from phd_visualizations.utils import rename_signal_ids_to_var_ids

%autoreload 2

# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

display(df.head())

df["Jmed_e"] = df['Jmed_b'] + df['Jmed_c'] + df['Jmed_d'] + df['Jmed_s_f']*1e-3 # kW

# Convert units to model units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

# Filter out nans until first value in Tts
start_idx = df.index.get_loc( df['Tts_c_b'].first_valid_index() )
df = df.iloc[start_idx:]
logger.info(f'Removed up to row {start_idx} to filter NaN values in thermal storage state')

display(df.head())

,FT-AQU-100,FT-AQU-101,TT-AQU-106,TT-AQU-107a,TT-AQU-109,CT-DES-001,FT-DES-003,FT-DES-005,SW2TC1,TE-DES-015,...,TT-SF-007,TT-SF-008,TT-SF-009,TT-SF-010,vfd_prod_power,PK-MED-E01-pa,vfd_brine_power,HW1TT21,vfd_sw_power,FT-DES-002_VFD
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:01+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:02+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:03+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:04+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,qmed_s,qts_dis,Tts_h_out,Tmed_s_in,Tts_c_b_in,wmed_f,qmed_f,qmed_d,Tmed_c_out,Tmed_c_in,...,Thx_p_out,Thx_s_in,Thx_s_out,qhx_p,qhx_s,T3wv_src,T3wv_dis_in,T3wv_dis_out,q3wv_src,q3wv_dis
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:31:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:32:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:33:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:34:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681


2024-02-20 09:14:48.200 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-02-20 09:14:48.202 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-02-20 09:14:48.202 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-02-20 09:14:48.203 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_out to C from C
2024-02-20 09:14:48.204 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_out to C from C
2024-02-20 09:14:48.205 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tsf_in to C from C
2024-02-20 09:14:48.206 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tsf_out to C from C
2024-02-20 09:14:48.207 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tts_h_t_in to C from C
2024-02-20 09:14:48.208 | DEBUG    | phd_visual

,qmed_s,qts_dis,Tts_h_out,Tmed_s_in,Tts_c_b_in,wmed_f,qmed_f,qmed_d,Tmed_c_out,Tmed_c_in,...,Thx_s_in,Thx_s_out,qhx_p,qhx_s,T3wv_src,T3wv_dis_in,T3wv_dis_out,q3wv_src,q3wv_dis,Jmed_e
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:44:00+00:00,0.03125,0.034011,17.423900,17.662380,26.683404,35.328423,0.010750,0.000816,16.069060,16.0,...,15.296492,16.923871,0.000000,0.0,96.424700,17.662380,26.683404,0.034011,0.03125,NaN
2023-10-30 07:45:00+00:00,0.03125,0.033959,17.432972,17.660400,26.681435,35.351805,0.010662,0.000818,16.067952,16.0,...,15.296910,16.952170,0.000000,0.0,96.422570,17.660400,26.681435,0.033959,0.03125,NaN
2023-10-30 07:46:00+00:00,0.03125,0.034200,17.430652,17.649843,26.670880,35.418997,0.010647,0.000802,16.031067,16.0,...,15.296857,17.009618,1.387997,0.0,96.415720,17.649843,26.670880,0.034200,0.03125,NaN
2023-10-30 07:47:00+00:00,0.03125,0.034135,17.429460,17.639787,26.663742,35.427615,0.010652,0.000808,16.057810,16.0,...,15.296970,17.080558,1.577559,0.0,96.413525,17.639787,26.663742,0.034135,0.03125,NaN
2023-10-30 07:48:00+00:00,0.03125,0.034047,17.432343,17.645888,26.655250,35.422630,0.010637,0.000806,16.035168,16.0,...,15.297030,17.168850,1.576460,0.0,96.412153,17.645888,26.655250,0.034047,0.03125,NaN


In [2]:
# Test model
from models_psa import SolarMED

%autoreload 2

# Initialize model

model = SolarMED(
    sample_time=sample_rate_numeric,
    curve_fits_path='data/curve_fits.json',
    
    # Initial states. Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[0], df['Tts_h_m'].iloc[0], df['Tts_h_b'].iloc[0]], 
    Tts_c=[df['Tts_c_t'].iloc[0], df['Tts_c_m'].iloc[0], df['Tts_c_b'].iloc[0]],
    Tsf_in=df['Tsf_in'].iloc[0],
    
    cost_w = cost_w, # €/m³ 
    cost_e = cost_e # €/kWhe
)

PydanticUserError: Field 'lims_T_hot' requires a type annotation

For further information visit https://errors.pydantic.dev/2.6/u/model-field-missing-annotation